In [37]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.cluster.vq import whiten
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import cv2

In [38]:
dataset_path = "./dataset/test_data"
image_paths = os.listdir(dataset_path)

In [39]:
if os.path.exists("Output"):

    for file in os.listdir("Output"): 
        file_path = os.path.join("Output", file)

        if os.path.isfile(file_path):
            os.unlink(file_path)
        
    for file in os.listdir("Output/No_Mask_Detected"): 
        file_path = os.path.join("Output/No_Mask_Detected", file)
        os.unlink(file_path)

else:
    os.makedirs("Output")
    os.makedirs("Output/No_Mask_Detected")

In [40]:
def dominant_color(image):

    scale = 75
    rows, cols = image.shape[:2]
    new_image = cv2.resize(image, (int(scale * float(cols) / rows), scale))

    pixels = new_image.reshape((-1, 3))

    r = new_image[:,:,2]
    g = new_image[:,:,1]
    b = new_image[:,:,0]

    r_scaled = whiten(r)
    r_scaled = np.reshape(r_scaled, (-1,))

    g_scaled = whiten(g)
    g_scaled = np.reshape(g_scaled, (-1,))
    
    b_scaled = whiten(b)
    b_scaled = np.reshape(b_scaled, (-1,))

    df = pd.DataFrame({"r_scaled": r_scaled, "g_scaled": g_scaled, "b_scaled": b_scaled})
    data = df[["r_scaled", "g_scaled", "b_scaled"]]
    
    scores = []
    num_clusters = range(2, 5)

    for k in num_clusters:

        try:
            model = KMeans(n_clusters=k, random_state=42)
            pred = model.fit_predict(data)
            scores.append(silhouette_score(data, pred))

        except ValueError:
            break

    if len(scores) == 0:
        return 0

    k = np.argmax(scores) + 2

    model = KMeans(n_clusters=k, random_state=42)
    model.fit(data)

    pixels_colorwise = np.unique(model.labels_, return_counts=True)[1]
    percentage = pixels_colorwise / pixels.shape[0]

    return max(percentage)
    

In [41]:
image_num = 1

for path in image_paths:

    print(f"Processing Image {image_num}...")

    image = cv2.imread(f"{dataset_path}/{path}")
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    blurred_image = cv2.GaussianBlur(gray_image, (3, 3), 0.1)

    edges = cv2.Canny(blurred_image, threshold1=20, threshold2=100)

    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    image_width, image_height = image.shape[:2]
    image_size = image_width * image_height

    count = 1

    counter = 1
    
    for cnt in contours:

        x, y, w, h = cv2.boundingRect(cnt)

        segment = image[y: y + h, x: x + w]
        segment_size = w * h

        if segment_size < 0.3 * image_size:
            continue

        dominance = dominant_color(segment)

        if dominance >= 0.4:            
            cv2.imwrite(f"./Output/{path}_{count}.png", segment)
            count += 1

    if count == 1:
        cv2.imwrite(f"./Output/No_Mask_Detected/{path}.png", image)
 
    print(f"Image {image_num} processed.")
    image_num += 1


Processing Image 1...
Image 1 processed.
Processing Image 2...
Image 2 processed.
Processing Image 3...
Image 3 processed.
Processing Image 4...
Image 4 processed.
Processing Image 5...
Image 5 processed.
Processing Image 6...
Image 6 processed.
Processing Image 7...
Image 7 processed.
Processing Image 8...
Image 8 processed.
Processing Image 9...
Image 9 processed.
Processing Image 10...
Image 10 processed.
Processing Image 11...
Image 11 processed.
Processing Image 12...
Image 12 processed.
Processing Image 13...
Image 13 processed.
Processing Image 14...
Image 14 processed.
Processing Image 15...
Image 15 processed.
Processing Image 16...
Image 16 processed.
Processing Image 17...
Image 17 processed.
Processing Image 18...
Image 18 processed.
Processing Image 19...
Image 19 processed.
Processing Image 20...
Image 20 processed.
